In [5]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import os

# --- Configuration Section ---

# 1. List of Major Forex Pairs to Download
# Most brokers use the standard 6-character names.
# Add or remove symbols as needed.
MAJOR_PAIRS = [
    "EURUSD", "GBPUSD", "USDJPY", "USDCHF", 
    "AUDUSD", "USDCAD", "NZDUSD"
]

# If your broker uses a suffix (e.g., 'EURUSD.m' or 'EURUSD_i'), 
# set it here. Otherwise, leave it as an empty string.
SYMBOL_SUFFIX = "" 

# 2. Timeframe for the Data
# Options: TIMEFRAME_M1, M5, M15, M30, H1, H4, D1, W1, MN1
TIMEFRAME = mt5.TIMEFRAME_D1  # Daily candles

# 3. Date Range for the Data (Last 10 Years)
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365*10)

# 4. Directory to Save the CSV Files
DATA_DIRECTORY = "forex_data_10_years"

# --- Main Script ---

def download_forex_data(symbol, timeframe, start_date, end_date):
    """
    Connects to MetaTrader 5, downloads historical data for a given symbol,
    and returns it as a pandas DataFrame.
    """
    print(f"\n--- Starting Download for {symbol} ---")

    # 1. Establish connection to the MetaTrader 5 terminal
    if not mt5.initialize():
        print(f"initialize() failed, error code = {mt5.last_error()}")
        print("\nCRITICAL ERROR: Could not connect to MetaTrader 5.")
        print("Ensure the MT5 terminal is running and 'Allow Algo Trading' is enabled.")
        return None
    
    # 2. Request the historical data
    print(f"Requesting data for {symbol} from {start_date.date()} to {end_date.date()}...")
    
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)

    if rates is None:
        print(f"No data received for {symbol}. Error: {mt5.last_error()}")
        print(f"Check if the symbol '{symbol}' is available in your Market Watch.")
        return None
    
    if len(rates) == 0:
        print(f"Data received, but it's empty for {symbol} in the specified date range.")
        return None

    # 3. Convert data to a pandas DataFrame for easier handling
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.rename(columns={
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'tick_volume': 'Volume'
    }, inplace=True)
    
    # Keep only the essential OHLCV columns
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    print(f"Successfully processed {len(df)} data points (candles) for {symbol}.")
    
    return df

def save_data_to_csv(df, symbol, timeframe_str, directory):
    """
    Saves the DataFrame to a CSV file in a specified directory.
    """
    if df is None or df.empty:
        print(f"Cannot save data for {symbol} because the DataFrame is empty.")
        return

    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

    # Construct a descriptive filename
    filename = f"{symbol}_{timeframe_str}_10_years.csv"
    filepath = os.path.join(directory, filename)
    
    # Save the DataFrame to a CSV file
    df.to_csv(filepath)
    
    print(f"Data for {symbol} successfully saved to: {filepath}")

def timeframe_to_string(timeframe):
    """Helper function to convert MT5 timeframe constant to a string."""
    timeframe_map = {
        mt5.TIMEFRAME_M1: "M1", mt5.TIMEFRAME_M5: "M5", mt5.TIMEFRAME_M15: "M15",
        mt5.TIMEFRAME_M30: "M30", mt5.TIMEFRAME_H1: "H1", mt5.TIMEFRAME_H4: "H4",
        mt5.TIMEFRAME_D1: "D1", mt5.TIMEFRAME_W1: "W1", mt5.TIMEFRAME_MN1: "MN1"
    }
    return timeframe_map.get(timeframe, "Unknown")

# --- Execute the script ---
if __name__ == "__main__":
    print("--- Forex Market Data Downloader ---")

    for pair in MAJOR_PAIRS:
        # Construct the full symbol name with any suffix
        symbol_name = pair + SYMBOL_SUFFIX
        
        # Download the data for the pair
        forex_df = download_forex_data(symbol_name, TIMEFRAME, START_DATE, END_DATE)
        
        # Save the data to a CSV file if download was successful
        if forex_df is not None:
            timeframe_str = timeframe_to_string(TIMEFRAME)
            save_data_to_csv(forex_df, symbol_name, timeframe_str, DATA_DIRECTORY)

    # Shut down the connection to MT5 after the loop finishes
    mt5.shutdown()
    print("\n--- All Downloads Finished. Disconnected from MetaTrader 5. ---")

--- Forex Market Data Downloader ---

--- Starting Download for EURUSD ---
Requesting data for EURUSD from 2015-07-24 to 2025-07-21...
Successfully processed 2693 data points (candles) for EURUSD.
Created directory: forex_data_10_years
Data for EURUSD successfully saved to: forex_data_10_years\EURUSD_D1_10_years.csv

--- Starting Download for GBPUSD ---
Requesting data for GBPUSD from 2015-07-24 to 2025-07-21...
Successfully processed 2693 data points (candles) for GBPUSD.
Data for GBPUSD successfully saved to: forex_data_10_years\GBPUSD_D1_10_years.csv

--- Starting Download for USDJPY ---
Requesting data for USDJPY from 2015-07-24 to 2025-07-21...
Successfully processed 2693 data points (candles) for USDJPY.
Data for USDJPY successfully saved to: forex_data_10_years\USDJPY_D1_10_years.csv

--- Starting Download for USDCHF ---
Requesting data for USDCHF from 2015-07-24 to 2025-07-21...
Successfully processed 2693 data points (candles) for USDCHF.
Data for USDCHF successfully saved to:

In [2]:
pip install MetaTrader5

   ---------------------------------------- 0.0/50.3 kB ? eta -:--:--
   -------------------------------- ------- 41.0/50.3 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 50.3/50.3 kB 852.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
